<a href="https://colab.research.google.com/github/georgematheww/AI-and-machine-learning/blob/main/Adversarial%20Intelligence%3A%20Comparative%20Robustness%20of%20AI-Based%20Threat%20Detection%20to%20Evasive%20Cyber%20Attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# FULL MINI-SCALE REPLICATION: CNN vs SVM Robustness
# ==============================================================================

# 1. INSTALL LIBRARIES
# ------------------------------------------------------------------------------
!pip install tensorflow scikit-learn pandas numpy matplotlib -q

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Input

# 2. DATA LOADING & PREPROCESSING
# ------------------------------------------------------------------------------
print(">>> Downloading NSL-KDD Dataset...")
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt -O KDDTrain.txt
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt -O KDDTest.txt

cols = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","class","difficulty"]

# Load CSVs
df_train = pd.read_csv('KDDTrain.txt', names=cols)
df_test = pd.read_csv('KDDTest.txt', names=cols)

def preprocess_mini_scale(df):
    """
    Simplified preprocessing for the mini-scale test:
    1. Maps 'normal' to 0 and attacks to 1.
    2. Drops categorical text columns to avoid complex encoding steps.
    """

    df['label'] = df['class'].apply(lambda x: 0 if x == 'normal' else 1)


    df = df.drop(['protocol_type', 'service', 'flag', 'class', 'difficulty'], axis=1)
    return df

print(">>> Preprocessing Data...")
df_train = preprocess_mini_scale(df_train)
df_test = preprocess_mini_scale(df_test)

# Extract X (features) and y (labels)
X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Scale Data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for CNN (Samples, Features, 1)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"    Train Shape: {X_train.shape}")
print(f"    Test Shape:  {X_test.shape}")

# 3. BUILD & TRAIN MODELS
# ------------------------------------------------------------------------------

# --- A. SVM (The "Robust" Baseline) ---
print("\n>>> Training SVM (This may take ~60 seconds)...")

svm = SVC(kernel='rbf', C=1.0)
svm.fit(X_train[:10000], y_train[:10000])
svm_acc = svm.score(X_test, y_test)
print(f"    SVM Clean Accuracy: {svm_acc * 100:.2f}%")

# --- B. CNN (The "Glass Cannon") ---
print("\n>>> Training CNN...")
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),       # Explicit Input Layer
    Conv1D(32, 3, activation='relu'),         # Convolution
    MaxPooling1D(2),                          # Pooling
    Flatten(),                                # Flatten to 1D
    Dense(64, activation='relu'),             # Dense Layer
    Dense(1, activation='sigmoid')            # Output (Binary)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_cnn, y_train, epochs=3, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate on clean data
loss, cnn_clean_acc = model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"    CNN Clean Accuracy: {cnn_clean_acc * 100:.2f}%")

# 4. ADVERSARIAL ATTACK (FGSM)
# ------------------------------------------------------------------------------
print("\n>>> Generating FGSM Attacks...")

def fgsm_attack_safe(model, images, labels, epsilon=0.1):
    """
    Generates Adversarial Examples using Fast Gradient Sign Method.
    FIXED: Explicitly handles Tensor/Numpy conversion for Keras 3 compatibility.
    """
    # 1. Convert to constant Tensors for the calculation
    images_tensor = tf.convert_to_tensor(images, dtype=tf.float32)
    labels_tensor = tf.convert_to_tensor(labels, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(images_tensor)
        predictions = model(images_tensor)
        loss = tf.keras.losses.binary_crossentropy(labels_tensor, predictions)

    # 2. Calculate Gradient
    gradient = tape.gradient(loss, images_tensor)

    # 3. Create Perturbation (Sign of Gradient)
    signed_grad = tf.sign(gradient)

    # 4. Apply Noise
    adversarial_images = images_tensor + (epsilon * signed_grad)

    return adversarial_images.numpy()

# Attack Parameters
epsilon = 0.1
sample_size = 1000

# Prepare sample data
X_sample = X_test_cnn[:sample_size]
y_sample = y_test[:sample_size].reshape(-1, 1).astype(np.float32)

# Generate the attack
X_adv = fgsm_attack_safe(model, X_sample, y_sample, epsilon=epsilon)

# Evaluate the CNN on the attacked data
loss_adv, cnn_adv_acc = model.evaluate(X_adv, y_sample, verbose=0)

# 5. FINAL REPORT (JOINT DISPLAY)
# ------------------------------------------------------------------------------
print("\n" + "="*50)
print("FINAL RESULTS: ROBUSTNESS GAP ANALYSIS")
print("="*50)
print(f"1. SVM Clean Accuracy:       {svm_acc * 100:.2f}%")
print(f"2. CNN Clean Accuracy:       {cnn_clean_acc * 100:.2f}%")
print(f"3. CNN Adversarial Accuracy: {cnn_adv_acc * 100:.2f}% (Under FGSM Attack)")
print("-" * 50)
gap = (cnn_clean_acc * 100) - (cnn_adv_acc * 100)
print(f"ROBUSTNESS GAP: {gap:.2f} percentage points")
print("="*50)
if gap > 20:
    print("CONCLUSION: The CNN collapsed under attack (High Gap).")
    print("This confirms the 'Glass Cannon' theory from the abstract.")
else:
    print("CONCLUSION: The CNN resisted the attack.")

>>> Downloading NSL-KDD Dataset...
--2025-12-04 22:58:35--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19109424 (18M) [text/plain]
Saving to: ‘KDDTrain.txt’

KDDTrain.txt        100%[===================>]  18.22M  --.-KB/s    in 0.05s   

2025-12-04 22:58:36 (402 MB/s) - ‘KDDTrain.txt’ saved [19109424/19109424]

--2025-12-04 22:58:36--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3